In [2]:
! pip install deepctr

     |████████████████████████████████| 122kB 5.3MB/s 


In [3]:
import pandas as pd
from deepctr.feature_column import SparseFeat,DenseFeat,VarLenSparseFeat,get_feature_names
from deepctr.models import DIN
import random
import datetime
import time
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
df_electronics=pd.read_csv('/content/gdrive/My Drive/dl_group/df_electronics_train.csv')
df_electronics=df_electronics[:1000]

In [10]:
import random
def model_attr(model_attr):
    if '&' in model_attr:
        result=model_attr.split("&")
        return random.choice(result)
    else:
        return model_attr
df_electronics['model_attr']=df_electronics['model_attr'].map(lambda x:model_attr(x))

def expand(df, col, sep=','):
    r = df[col].str.split(sep)
    d = {c: df[c].values.repeat(r.str.len(), axis=0) for c in df.columns}
    d[col] = [i for sub in r for i in sub]
    return pd.DataFrame(d)
df_electronics=expand(df_electronics, 'category', '&')

def category(category):
    if '&' not in category:
        return category+"&"+"other"
    else:
        return category

def user_attr_2int(user_attr,user_attr_dict):
    if user_attr in user_attr_dict:
        return user_attr_dict[user_attr]
    else:
        return random.choice(list(user_attr_dict.values()))
def model_attr2int(model_attr,model_attr_dict):
    if model_attr in model_attr_dict:
        return model_attr_dict[model_attr]
    else:
        return random.choice(list(model_attr_dict.values()))


def brand_2int(brand_2,brand_dict):
    if brand_2 in brand_dict:
        return brand_dict[brand_2]
    else:
        return random.choice(list(brand_dict.values()))

def category_12int(category_1,category_1_dict):
    if category_1 in category_1_dict:
        return category_1_dict[category_1]
    else:
        return random.choice(list(category_1_dict.values()))
def category_22int(category_2,category_2_dict):
    if category_2 in category_2_dict:
        return category_2_dict[category_2]
    else:
        return random.choice(list(category_2_dict.values()))

def item_id_2int(item_id, item_id_dict):
    if item_id in item_id_dict:
        return item_id_dict[item_id]
    else:
        return random.choice(list(item_id_dict.values()))

def user_id_2int(user_id, user_id_dict):
    if user_id in user_id_dict:
        return user_id_dict[user_id]
    else:
        return random.choice(list(user_id_dict.values()))

item_id_set=set(df_electronics['item_id'].unique())
item_id_dict={}
for index,cate in enumerate(item_id_set):
    item_id_dict[cate]=index

user_id_set=set(df_electronics['user_id'].unique())
user_id_dict={}
for index,cate in enumerate(user_id_set):
    user_id_dict[cate]=index


model_attr_set=set(df_electronics['model_attr'].unique())
model_attr_dict={}
for index,gender in enumerate(model_attr_set):
    model_attr_dict[gender]=index
category_set=set(df_electronics['category'].unique())
category_dict={}
for index,cate in enumerate(category_set):
    category_dict[cate]=index

brand_set=set(df_electronics['brand'].unique())
brand_dict={}
for index,cate in enumerate(brand_set):
    brand_dict[cate]=index

user_attr_set=set(df_electronics['user_attr'].unique())
user_attr_dict={}
for index,cate in enumerate(user_attr_set):
    user_attr_dict[cate]=index

item_id_set=set(df_electronics['item_id'].unique())
item_id_dict={}
for index,cate in enumerate(item_id_set):
    item_id_dict[cate]=index


df_electronics['model_attr']=df_electronics['model_attr'].map(lambda x:model_attr2int(x,model_attr_dict))
df_electronics['category']=df_electronics['category'].map(lambda x:category_12int(x,category_dict))
df_electronics['brand']=df_electronics['brand'].map(lambda x:brand_2int(x,brand_dict))
df_electronics['user_attr']=df_electronics['user_attr'].map(lambda x:user_attr_2int(x,user_attr_dict))
df_electronics['item_id']=df_electronics['item_id'].map(lambda x:item_id_2int(x,item_id_dict))
df_electronics['user_id']=df_electronics['user_id'].map(lambda x:user_id_2int(x,user_id_dict))



In [11]:
def convert(x):
    d = datetime.datetime.strptime(x,"%Y-%m-%d")
    t = d.timetuple()
    timeStamp = int(time.mktime(t))
    return timeStamp
###################################################
df_electronics['timestamp'] = df_electronics.timestamp.apply(lambda x: convert(x))
df_electronics=df_electronics.sort_values(by="timestamp", ascending=True)
itemprofile=df_electronics[['category','brand','item_id']].to_dict(orient='records')
electronics_profile=df_electronics.to_dict(orient='records')
user_df=df_electronics.drop_duplicates("user_id")
groupdata=user_df.groupby("user_id")
user_behavior=[]
max=0
for groupname,data in groupdata:
    userkey=groupname
    data=data.to_dict(orient="records")
    item_id={item['item_id'] for item  in data}
    items=list(item_id)
    length = len(items)

    allitems=set(item_id_dict.values())

    seleted1 = set(random.sample(allitems, 10))
    otheritems=allitems.difference(seleted1)
    if length>max:
        max=length
    getnum=10
    seleted2=[]
    seleted2.extend(items)
    samplenum=getnum-length
    if samplenum>0:
        seleted = set(random.sample(otheritems, samplenum))
        seleted2.extend(list(seleted))

    behavior = []

    for item in seleted1:
        if len(behavior) <10:
            userdict = {}
            behavior.append(item)
            hist_len = len(behavior)
            behavior1 = []
            behavior1.extend(behavior)
            if len(seleted2)<1:
                continue
            else:
                itemid = random.choice(seleted2)
                seleted2.remove(itemid)

                for cuitem in electronics_profile:
                    #'item_id', 'user_id', 'rating', 'timestamp', 'size', 'user_attr','model_attr', 'category', 'brand'
                    if cuitem["user_id"]==groupname:
                        userdict['model_attr']=cuitem['model_attr']
                        userdict['user_attr']=cuitem['user_attr']
                        userdict['user_id'] = groupname
                        if cuitem['item_id']==itemid:
                            userdict['brand']=cuitem['brand']
                            userdict['category']=cuitem['category']
                            userdict['rating'] = cuitem['rating']
                            userdict['item_id'] = itemid
                        else:
                            for item in itemprofile:
                                if item["item_id"] == itemid:
                                    userdict['brand'] = item['brand']
                                    userdict['category'] = item['category']
                                    userdict['rating'] = 0
                                    userdict['item_id'] = itemid

            if len(behavior1) < 10:
               num = 10 - len(behavior1)
               for i in range(num+1):
                   behavior1.append(0)
               userdict["hist_movie_id"]= " ".join((str(id) for id in behavior1))
               userdict['hist_len']=len(behavior)
               user_behavior.append(userdict)
               continue

        else:
            userdict = {}
            hist_len = 10
            if len(seleted2) < 1:
                continue
            else:
             behavior1 = []
             behavior1.extend(behavior)
             itemid = random.choice(seleted2)
             seleted2.remove(itemid)
             for cuitem in electronics_profile:
                 # 'item_id', 'user_id', 'rating', 'timestamp', 'size', 'user_attr','model_attr', 'category', 'brand'
                 if cuitem["user_id"] == groupname:
                     userdict['model_attr'] = cuitem['model_attr']
                     userdict['user_attr'] = cuitem['user_attr']
                     userdict['user_id'] = groupname
                     if cuitem['item_id'] == itemid:
                         userdict['brand'] = cuitem['brand']
                         userdict['category'] = cuitem['category']
                         userdict['item_id'] = itemid
                         userdict['rating'] = cuitem['rating']
                     else:
                         for item in itemprofile:
                             if item["item_id"] == itemid:
                                 userdict['brand'] = item['brand']
                                 userdict['category'] = item['category']
                                 userdict['rating'] = 0
                                 userdict['item_id'] = itemid
                 userdict["hist_movie_id"] = " ".join((str(id) for id in behavior1))
                 userdict['hist_len'] = len(behavior)
                 user_behavior.append(userdict)
                 continue

finalResult=pd.DataFrame(user_behavior)
finalResult.to_csv('electtrain.csv',sep='\t')
#
# all_data_dict=df_movies.to_dict(orient="records")
#
# user_behavior_df=pd.DataFrame(user_behavior)


print("preprocess done !!!")

preprocess done !!!
